In [1]:
import logging

logger = logging.getLogger()

# format
formatter = logging.Formatter('[%(asctime)s] [%(levelname)s] %(message)s')

# add stream handler
handler = logging.StreamHandler(sys.stdout)                             
handler.setFormatter(formatter)
logger.addHandler(handler)

# enable all logging level
logger.setLevel(logging.INFO)

Definition for the board.

In [2]:
import os

os.environ["OKFP_ROOT"] = "C:/Users/sean9/Downloads/FrontPanelUSB"

from ece437.ok import OKFrontPanel

path = "C:/Users/sean9/ECE437/Final/Final.runs/impl_1/final_top.bit"
fp = OKFrontPanel(firmware_path=path)

Start talking to LSM303. Benchmark speed.

In [3]:
from ece437.presets import I2C_1_ENDPOINTS
from ece437.i2c import I2CController
from ece437.sensor import LSM303Accelerometer, LSM303Magnetometer
from IPython.display import clear_output
import time
from datetime import datetime

with fp, I2CController(fp, I2C_1_ENDPOINTS) as i2c1:
    print(f"serial={fp.serial_number}")

    acc = LSM303Accelerometer(i2c1)
    acc.configure() # defaults

    mag = LSM303Magnetometer(i2c1)
    mag.configure() # defaults

    t0 = datetime.now()

    for _ in range(100):
        x, y, z = acc.read()
        #print(f'ACC: x={x:.03f}, y={y:.03f}, z={z:.03f}')

        #x, y, z = mag.read()
        #print(f'MAG: x={x:.03f}, y={y:.03f}, z={z:.03f}')    

        #clear_output(wait=True)
    t1 = datetime.now()

    delta = t1 - t0
    dt = delta.total_seconds() * 1000
    logger.info(f't_sensor={dt:.3f} ms')

[2022-12-15 15:35:30,905] [INFO] found 1 devices on this system
[2022-12-15 15:35:30,911] [INFO] upload firmware 'C:/Users/sean9/ECE437/Final/Final.runs/impl_1/final_top.bit'
serial=1911000P3V
[2022-12-15 15:35:31,461] [INFO] t_sensor=397.102 ms


Acquire noise dataset.

In [22]:
from ece437.presets import I2C_1_ENDPOINTS
from ece437.i2c import I2CController
from ece437.sensor import LSM303Accelerometer, LSM303Magnetometer
from IPython.display import clear_output
import time
from datetime import datetime
import numpy as np
with fp, I2CController(fp, I2C_1_ENDPOINTS) as i2c1:
    print(f"serial={fp.serial_number}")

    acc = LSM303Accelerometer(i2c1)
    acc.configure() # defaults

    mag = LSM303Magnetometer(i2c1)
    mag.configure() # defaults

    t_hist = []
    acc_hist = []
    mag_hist = []
    for _ in range(1000):
        t_hist.append(datetime.now())
        acc_hist.append(acc.read())
        mag_hist.append(mag.read())

    # convert timestamp to millisecond
    t_hist = [t.timestamp() * 1000 for t in t_hist]
    t0 = t_hist[0]
    t_hist = [t - t0 for t in t_hist]

    t_hist = np.array(t_hist)
    acc_hist = np.array(acc_hist)
    mag_hist = np.array(mag_hist)

    data = np.hstack([t_hist[:, np.newaxis], acc_hist, mag_hist])

[2022-12-15 15:43:56,239] [INFO] found 1 devices on this system
[2022-12-15 15:43:56,247] [INFO] upload firmware 'C:/Users/sean9/ECE437/Final/Final.runs/impl_1/final_top.bit'
serial=1911000P3V


In [23]:
np.savetxt("sensor_noise.csv", data, delimiter=",")